In [87]:
import numpy as np
import math
import pandas as pd
from itertools import permutations

# Markov Models
<br>
p(after|before)

In [3]:
sequences = ["ABABAB","ABABAA","BBABAA","AAB"]

In [4]:
def pi(seq, letter):
    count = 0
    for s in seq:
        if s[0]==letter:
            count+=1
    if(count == 0 or count < 0):
        print(letter,"not in the sequences")
        return None
    else:
        print(str(count)+"/"+str(len(seq)))
        return count/len(seq)

In [5]:
pi(sequences,'A')

3/4


0.75

In [6]:
pi(sequences,'B')

1/4


0.25

In [16]:
A = [0.7,0.4,0.3]
B = [0.3,0.6,0.7]
df = pd.DataFrame({"A":A, "B":B}, index = ["prevA","prevB","pi"])
df

,A,B
prevA,0.7,0.3
prevB,0.4,0.6
pi,0.3,0.7


In [17]:
df["A"]["prevA"]

0.7

In [26]:
def probability(df, find_seq):
    first = find_seq[0]
    p = df[first]["pi"]
    print("{}x".format(p),end = "")
    for _ in range(len(find_seq)-1):
        before_idx = _
        after_idx = _+1
        print("{}x".format(df[find_seq[after_idx]]["prev"+find_seq[before_idx]]),end = "")
        p*=df[find_seq[after_idx]]["prev"+find_seq[before_idx]]
    return p

what is the probability of AABBABAB

In [28]:
probability(df,"AABBABAB")

0.3x0.7x0.3x0.6x0.4x0.3x0.4x0.3x

0.00054432

# Hidden Markov Model: 
# cannot observe states(A,B) can observe outputs(a,b)

In [116]:
A = [0.3,0.7,0.3,0.7,0.4,]
B = [0.5,0.5,0.7,0.3,0.6]
df = pd.DataFrame({"A":A, "B":B}, index = ["a","b","pi","prevA","prevB"])
df

,A,B
a,0.3,0.5
b,0.7,0.5
pi,0.3,0.7
prevA,0.7,0.3
prevB,0.4,0.6


In [29]:
def split(word):
    return [char for char in word]

In [117]:
def hmm_output(df, find, observed):
    percents = []
    observed = split(observed)
    find = split(find)
    p = 1
    for o, f in zip(observed, find):
        print("p({}|{}) x ".format(f,o),end = "")
        percents.append(str(df[o][f]))
        p*=df[o][f]
    print("\n")
    print(" x ".join(percents))
    return p

We observe the AABBABAB.<br>
What is the probability of abababab?

$p(o_1,o_2, \cdots o_t|q_1,q_2,\cdots q_t)$

In [118]:
hmm_output(df,"abababab","AABBABAB")#p(abababab|AABBABAB)

p(a|A) x p(b|A) x p(a|B) x p(b|B) x p(a|A) x p(b|B) x p(a|A) x p(b|B) x 

0.3 x 0.7 x 0.5 x 0.5 x 0.3 x 0.5 x 0.3 x 0.5


0.00118125

we observe abababab<br>
what is the probability of AABBABAB?<br>

$p(q_1,q_2,\cdots q_t|o_1,o_2, \cdots o_t) = \frac{p(o_1,o_2, \cdots o_t|q_1,q_2 \cdots q_t)p(q_1,q_2 \cdots q_t)}{p(o_1,o_2, \cdots o_t)}$

In [144]:
def p_state(model, letter):#분모 구하기
    states = np.unique(df.columns)
    
    if len(letter) == 1:
        p = 0
        process = []
        for s in list(states):
            print("p({}|{})p({}) + ".format(letter, s, s),end = "")
            tmp_p = df[s][letter]
            process.append(str(tmp_p))
            tmp_p *= df[s]["pi"]
            process.append(str(df[s]["pi"]))
            p += tmp_p
        print("\n")
        print("x".join(process))
        return p
    else:
        p = 0
        tmp_states = []
        states = list(states)
        for _ in range(len(states)):
            tmp_states += states
        #print(tmp_states)
        tmp_permute = permutations(tmp_states,len(letter))
        permute = []
        for per in tmp_permute:
            permute.append("".join(per))
        permute = np.unique(permute)
        tmp_p = 0
        for per in permute:
            
            tmp_p = hmm_output(df,letter,per)*probability(df,per)
            print("\n\n>>p({}|{})p({}): {}".format(letter,per,per,tmp_p))
            p += tmp_p
        return p

In [145]:
p_state(df, 'a')

p(a|A)p(A) + p(a|B)p(B) + 

0.3x0.3x0.5x0.7


0.43999999999999995

In [146]:
p_state(df,'aa')

p(a|A) x p(a|A) x 

0.3 x 0.3
0.3x0.7x

>>p(aa|AA)p(AA): 0.0189
p(a|A) x p(a|B) x 

0.3 x 0.5
0.3x0.3x

>>p(aa|AB)p(AB): 0.0135
p(a|B) x p(a|A) x 

0.5 x 0.3
0.7x0.4x

>>p(aa|BA)p(BA): 0.041999999999999996
p(a|B) x p(a|B) x 

0.5 x 0.5
0.7x0.6x

>>p(aa|BB)p(BB): 0.105


0.1794

In [147]:
p_state(df,'aab')#굉장히 귀찮고 어려움: 교수님 피셜

p(a|A) x p(a|A) x p(b|B) x 

0.3 x 0.3 x 0.5
0.3x0.7x0.3x

>>p(aab|AAB)p(AAB): 0.002835
p(a|A) x p(a|B) x p(b|A) x 

0.3 x 0.5 x 0.7
0.3x0.3x0.4x

>>p(aab|ABA)p(ABA): 0.0037799999999999995
p(a|A) x p(a|B) x p(b|B) x 

0.3 x 0.5 x 0.5
0.3x0.3x0.6x

>>p(aab|ABB)p(ABB): 0.00405
p(a|B) x p(a|A) x p(b|A) x 

0.5 x 0.3 x 0.7
0.7x0.4x0.7x

>>p(aab|BAA)p(BAA): 0.020579999999999998
p(a|B) x p(a|A) x p(b|B) x 

0.5 x 0.3 x 0.5
0.7x0.4x0.3x

>>p(aab|BAB)p(BAB): 0.006299999999999999
p(a|B) x p(a|B) x p(b|A) x 

0.5 x 0.5 x 0.7
0.7x0.6x0.4x

>>p(aab|BBA)p(BBA): 0.0294


0.06694499999999999

In [148]:
def p_real(df, observe, find):
    #분자 계산
    up = hmm_output(df,observe, find)*probability(df,find)
    #분모:
    down = p_state(df,observe)
    print("=======================")
    print("result : {:.4f}".format(up/down))
    return up/down

In [149]:
p_real(df,"abababab","AABBABAB")

p(a|A) x p(b|A) x p(a|B) x p(b|B) x p(a|A) x p(b|B) x p(a|A) x p(b|B) x 

0.3 x 0.7 x 0.5 x 0.5 x 0.3 x 0.5 x 0.3 x 0.5
0.3x0.7x0.3x0.6x0.4x0.3x0.4x0.3x=======================
result : inf


<ipython-input-148-941ad9d94fec>:7: RuntimeWarning: divide by zero encountered in double_scalars
  print("result : {:.4f}".format(up/down))
<ipython-input-148-941ad9d94fec>:8: RuntimeWarning: divide by zero encountered in double_scalars
  return up/down


inf

In [5]:
def uniqueletters(seq):
    tmp = []
    for s in seq:
        tmp = tmp+split(s)
    return list(set(tmp))
        

In [6]:
uniqueletters(sequences)

['A', 'B']

In [7]:
def total_slices(seq):
    total_count = 0
    for s in seq:
        
        total_count+=len(s)-1
    return total_count

In [8]:
def p(seq,command):
    #print(split(command))
    if '|' in split(command):
        start,end = command.split("|")
        letters = uniqueletters(seq)
        real_c = 0
        total_ = 0
        for l in letters:
            tmp_command = end+"->"+l
            print(tmp_command)
            c = p(seq,tmp_command)
            
            total_ += c
            if start == l:
                real_c = c
        
        
        print(real_c,"/",total_)
        return real_c/total_
            
    elif '-' in split(command) and '>' in split(command):#before->after
        start,end = command.split("->")
        count = 0
        total = total_slices(seq)
        for s in seq:
            for _ in range(len(s)-1):
                first = s[_]
                second = s[_+1]
                
                if first == start and second == end:
                    count+=1
        print(count)
        return count
                    
    else:
        print("wrong command")

In [9]:
p(sequences,'A->B')

7


7

In [10]:
p(sequencesces, 'B->A')

6


6

In [108]:
p(sequences,'A->A')

3


3

In [12]:
p(sequences,'B|A')

A->A
3
A->B
7
7 / 10


0.7

In [14]:
p(sequences, 'A|B')

B->A
6
B->B
1
6 / 7


0.8571428571428571

In [15]:
p(sequences, 'B|A')

A->A
3
A->B
7
7 / 10


0.7